## Data and imports

In [27]:
import pandas as pd
import altair as alt

In [28]:
url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv'
df = pd.read_csv(url)

In [29]:
df.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


In [30]:
df.shape

(8862, 22)

In [31]:
df['Year Constructed'].unique()

array([1975, 2004, 2000, 1999, 1980, 1997, 1976, 1994, 2001, 1993, 1970,
       1977, 2002, 2006, 1990, 1988, 1998, 1981, 1995, 1890, 1894, 1896,
       1987, 1985, 1753, 1841, 1972, 1973, 1860, 1986, 1930, 1966, 1946,
       1960, 1942, 1983, 1996, 1978, 1979,    0, 1982, 1984, 1974, 1989,
       1870, 1855, 1991, 1913, 1842, 1859, 2003, 1947, 1926, 1910, 1949,
       1963, 1964, 1968, 1929, 1917, 1967, 1920, 2005, 1885, 1940, 1950,
       1958, 1961, 1971, 1992, 1954, 1932, 1900, 1867, 1956, 1965, 1953,
       1907, 1935, 1938, 1951, 1840, 1962, 2010, 2007, 2011, 1863, 1861,
       1955, 1936, 1934, 1880, 2008, 1959, 1952, 1957, 1948, 1927, 1945,
       1944, 1895, 1933, 1921, 1939, 1925, 1875, 1876, 1969, 1931, 1903,
       1848, 1918, 1922, 1905, 1915, 1886, 1832, 1908, 1916, 1919, 1937,
       1941, 1810, 1883, 1943, 1852, 1847, 1856, 1878, 1924, 1802, 1844,
       1850, 1838, 1891, 1854, 1843, 1862, 1898, 1901, 1845, 1904, 1839,
       1923, 1837, 1865, 1928, 1872, 1899, 1906, 18

Filter out buildings with Year Constructed = 0

In [32]:
df = df[(df['Year Constructed'] > 1800) & (df['Year Constructed'] <= 2025)]

In [33]:
df['Square Footage'].value_counts

<bound method IndexOpsMixin.value_counts of 0         144
1         144
2         144
3         144
4         144
        ...  
8853      760
8854      760
8859      130
8860    49552
8861      288
Name: Square Footage, Length: 8567, dtype: int64>

Buildings with zero reported square footage are outliers. While they remain important for data analytics, we can exclude them from visualizations to avoid skewing the results.

In [34]:
df = df[df['Square Footage'] > 0]


creating a new column - age of building 

In [35]:
df['Building Age'] = 2025 - df['Year Constructed']

In [36]:
df['Agency Name'].value_counts()

Agency Name
Department of Natural Resources              3031
Department of Corrections                    1406
Department of Transportation                 1085
Department of Human Services                  611
University of Illinois                        524
Southern Illinois University                  409
Historic Preservation Agency                  277
Department of Agriculture                     226
Department of Military Affairs                212
Department of Juvenile Justice                120
Department of State Police                    109
Illinois State University                     102
Department of Veterans' Affairs                91
Northern Illinois University                   74
Department of Central Management Services      60
Western Illinois University                    42
Office of the Secretary of State               41
Eastern Illinois University                    35
Northeastern Illinois University               18
Chicago State University              

In [37]:
agencies = [
            'University of Illinois',
            'Southern Illinois University',
            'Illinois State University']

In [38]:
df_viz1 = df[df['Agency Name'].isin(agencies)].copy()


In [39]:
df_viz1.shape

(1035, 23)

## Building Age vs. Square Footage by Agency

This interactive scatterplot explores the relationship between building age and size across different state agencies. 

- X-axis: Year Constructed (temporal encoding)
- Y-axis: Square Footage (quantitative encoding, log scale for better distribution)
- Color: Agency Name (categorical encoding using color)
- Interactivity: Hovering reveals detailed tooltips with building information
- Interactivity: Click and drag to zoom into specific areas, double-click to reset

The log scale on the y-axis helps visualize the wide range of building sizes more effectively.

In [40]:
agency_selection = alt.selection_point(fields=['Agency Name'], bind='legend')

scatter = alt.Chart(df_viz1).mark_circle().encode(
    x=alt.X('Building Age:Q', 
            axis=alt.Axis(title='Building Age')),
    y=alt.Y('Square Footage:Q', 
            scale=alt.Scale(type='log'),
            axis=alt.Axis(title='Square Footage (log scale)')),
    color=alt.Color('Agency Name:N', 
                    scale=alt.Scale(),
                    legend=alt.Legend(title='Agency')),
    tooltip=[
        alt.Tooltip('Location Name:N', title='Building'),
        alt.Tooltip('Agency Name:N', title='Agency'),
        alt.Tooltip('Building Age:Q', title='Building Age'),
        alt.Tooltip('Square Footage:Q', title='Square Feet'),
        alt.Tooltip('Usage Description:N', title='Usage'),
        alt.Tooltip('City:N', title='City'),
        alt.Tooltip('Bldg Status:N', title='Status')
    ],
    opacity=alt.condition(agency_selection, alt.value(1), alt.value(0.2))

).properties(
    width=700,
    height=400,
    title='Illinois State University Buildings: Age vs. Size by Agency'
).add_params(agency_selection)

scatter

alt.Chart(...)

Users can click on agency names in the legend to highlight specific agencies while fading others to 20% opacity. This lets analysis of individual agencies easier to identify patterns in building age and size for specific state departments.

In [41]:
#scatter.save('assets/json/building_age_size_scatter.json')

##  Interactive Dashboard - Building Usage Analysis

This dashboard allows exploration of building usage types and their size distributions. The visualization consists of:

Left Panel - Bar Chart:
- X-axis: Count of buildings (quantitative)
- Y-axis: Usage Description (categorical, sorted by count)
- Color: categorical encoding

Right Panel - Histogram:
- X-axis: Square Footage binned into ranges
- Y-axis: Count of buildings

Interactivity: 
- Brushing and Linking: Click and drag on the bar chart to select specific usage types
- The histogram dynamically updates to show only the square footage distribution of selected building types
- This allows you to compare size patterns across different usage categories
- Click on empty space to reset selection

Remove extreme outliers for better visualization

In [42]:
df_viz2 = df[df['Square Footage'] < 2000].copy() 

In [43]:
brush = alt.selection_interval(encodings=['y'])

In [44]:
usage_chart = alt.Chart(df_viz2).mark_bar().encode(
    x=alt.X('count():Q', 
            axis=alt.Axis(title='Number of Buildings')),
    y=alt.Y('Usage Description:N', 
            sort='-x',
            axis=alt.Axis(title='Building Usage Type')),
    color=alt.condition(
        brush,
        alt.Color('count():Q', scale=alt.Scale(), legend=None),
        alt.value('lightgray')
    ),
    tooltip=[
        alt.Tooltip('Usage Description:N', title='Usage Type'),
        alt.Tooltip('count():Q', title='Building Count')
    ]
).properties(
    width=300,
    height=400,
    title='Building Count by Usage Type'
).add_params(
    brush
)

In [45]:
size_histogram = alt.Chart(df_viz2).mark_bar().encode(
    x=alt.X('Square Footage:Q', 
            bin=alt.Bin(maxbins=30),
            axis=alt.Axis(title='Square Footage', format=',')),
    y=alt.Y('count():Q',
            axis=alt.Axis(title='Number of Buildings')),
    tooltip=[
        alt.Tooltip('Square Footage:Q', bin=True, title='Size Range', format=','),
        alt.Tooltip('count():Q', title='Building Count')
    ]
).properties(
    width=350,
    height=400,
    title='Square Footage Distribution (Select usage types to filter)'
).transform_filter(
    brush
)

In [46]:
dashboard = (usage_chart | size_histogram).properties(
    title=alt.TitleParams(
        'Interactive Dashboard: Building Usage and Size Analysis',
        fontSize=16,
        anchor='middle'
    )
)

dashboard

alt.HConcatChart(...)

In [47]:
#dashboard.save('assets/json/building_usage_dashboard.json')

## Summary


### Data Transformations:
1. Filtered out invalid years (Year Constructed = 0 or > 2025)
2. Removed buildings with zero square footage
3. For Visualization 1: Filtered to top 6 agencies for clarity
4. For Visualization 2: Removed extreme outliers (>2,000 sq ft) for better binning
5. Created 'Building Age' calculated field


### Encoding Choices:


#### Visualization 1 - Scatter Plot:
- Used log scale for Square Footage to handle wide range of values
- Interactive tooltips provide detailed information on hover
- Pan/zoom interactivity allows detailed exploration

#### Visualization 2 - Dashboard:
- Sorted usage types by count for easy comparison
- Brushing/linking enables dynamic filtering between charts
- Selected items remain colored while unselected turn gray
- Binned square footage (30 bins) creates readable histogram